<a href="https://colab.research.google.com/github/daspartho/fastai-part2/blob/main/backprop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Setup

In [22]:
import pickle,gzip,math,os,time,shutil,torch,matplotlib as mpl, numpy as np
from pathlib import Path
from torch import tensor
from urllib.request import urlretrieve
from fastcore.test import test_close
torch.manual_seed(42)

mpl.rcParams['image.cmap'] = 'gray'
torch.set_printoptions(precision=2, linewidth=125, sci_mode=False)
np.set_printoptions(precision=2, linewidth=125)

MNIST_URL='https://github.com/mnielsen/neural-networks-and-deep-learning/blob/master/data/mnist.pkl.gz?raw=true'
path_data = Path('data')
path_data.mkdir(exist_ok=True)
path_gz = path_data/'mnist.pkl.gz'

if not path_gz.exists():
    urlretrieve(MNIST_URL, path_gz)

with gzip.open(path_gz, 'rb') as f: 
    ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
x_train, y_train, x_valid, y_valid = map(tensor, [x_train, y_train, x_valid, y_valid])

### Foundations version

#### Basic architecture

In [23]:
n,m = x_train.shape
c = x_train.max()+1
n,m,c

(50000, 784, tensor(2.00))

In [24]:
nh=50 # number of hidden

In [25]:
w1 = torch.randn(m, nh)
b1 = torch.zeros(nh)
w2 = torch.randn(nh, 1)
b2 = torch.zeros(1)

In [26]:
def lin(x, w, b):
    return x@w + b

In [27]:
t = lin(x_valid, w1, b1)
t, t.shape

(tensor([[ -0.09,  11.87, -11.39,  ...,   5.48,   2.14,  15.30],
         [  5.38,  10.21, -14.49,  ...,   0.88,   0.08,  20.23],
         [  3.31,   0.12,   3.10,  ...,  16.89,  -6.05,  24.74],
         ...,
         [  4.01,  10.35, -11.25,  ...,   0.23,  -5.30,  18.28],
         [ 10.62,  -4.27,  10.72,  ...,  -2.87,  -2.87,  18.23],
         [  2.84,  -0.22,   1.43,  ...,  -3.91,   5.75,   2.12]]),
 torch.Size([10000, 50]))

In [28]:
def relu(x):
    return x.clamp_min(0.)

In [29]:
t = relu(t)
t

tensor([[ 0.00, 11.87,  0.00,  ...,  5.48,  2.14, 15.30],
        [ 5.38, 10.21,  0.00,  ...,  0.88,  0.08, 20.23],
        [ 3.31,  0.12,  3.10,  ..., 16.89,  0.00, 24.74],
        ...,
        [ 4.01, 10.35,  0.00,  ...,  0.23,  0.00, 18.28],
        [10.62,  0.00, 10.72,  ...,  0.00,  0.00, 18.23],
        [ 2.84,  0.00,  1.43,  ...,  0.00,  5.75,  2.12]])

In [30]:
def model(x):
    l1 = lin(x, w1, b1)
    l2 = relu(l1)
    return lin(l2, w2, b2)

In [31]:
res = model(x_valid)
res, res.shape

(tensor([[  25.75],
         [ -13.06],
         [-114.79],
         ...,
         [ -67.44],
         [ -74.48],
         [ -60.19]]), torch.Size([10000, 1]))